# Signal Chain Reconstruction

In [2]:
import torch
from torch import nn
import torchaudio
import os
import numpy as np
import matplotlib.pyplot as plt

### Load dataset

In [3]:
from src.gtfxdataset import GtFxDataset

AUDIO_DIR = "_assets/DATASET/GT-FX-TEST/"
ANNOTATIONS_FILE = os.path.join(AUDIO_DIR, "annotation.csv")

SAMPLE_RATE = 22050
NUM_SAMPLES = 22050*3

EFFECT_MAP = ["distortion", "chorus", "tremolo", "delay", "reverb"]
EFFECT = 1

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=512,
    hop_length=1050,
    n_mels=64,
    # power=2
)

spectrogram = torchaudio.transforms.Spectrogram(
    power=2,
    n_fft=127,
    win_length= 127,
    hop_length= 1040,
    normalized=True
)

mfcc = torchaudio.transforms.MFCC(
    sample_rate = SAMPLE_RATE, 
    n_mfcc = 64,
    melkwargs = {
        "n_fft": 1024,
        "hop_length": 1030,
        "n_mels": 64,
        "center": False})

fxData = GtFxDataset(ANNOTATIONS_FILE,
                        AUDIO_DIR,
                        mfcc,
                        SAMPLE_RATE,
                        NUM_SAMPLES,
                        device)

signal, _, _, _, _ = fxData[0]
print(f"There are {len(fxData)} samples in the dataset.")
print(f"Shape of signal: {signal.shape}")
    

Using device cpu
There are 22506 samples in the dataset.
Shape of signal: torch.Size([1, 64, 64])


#### Reconstruct Singal Chain

In [8]:
from src.classifier import model as cModel
from src.classifier import train as cTrain
from src.extrector import model as eModel
from src.extrector import train as eTrain
from src.effectapplier import EffectApplier
from src.util import play_audio
import pandas as pd
import random

_WEIGHTS_DIR = "_weights/"
_DRY_SAMPLE_DIR = "_assets/DATASET/test"

CLASSIFY_WEIGHTS = os.path.join(_WEIGHTS_DIR, "c55_classify.pth")

index = random.randint(0, len(fxData))
test_data = fxData[index]

sample_file = fxData.get_audio_sample_filename(index) + ".wav"
dry_file = sample_file[:2] + "0" + sample_file[3:sample_file.find('_', 4)] + ".wav"

print(f"Test file: {sample_file}", end="")
for i in range(5):
    if test_data[1][i] > 0.0:
        val = round(test_data[3][i].item(), 2)
        print(f", has {EFFECT_MAP[i]} with value {val}", end="")
print("")


classify = cModel.Classifier().to(device)
classify.load_state_dict(torch.load(CLASSIFY_WEIGHTS))

log = cTrain.test_single(classify, test_data)

extract = eModel.Extractor().to(device)
eff = EffectApplier(_DRY_SAMPLE_DIR, dry_file)

for fx, state in enumerate(log):
    if state == 1:
        WEIGHTS_FILE = os.path.join(_WEIGHTS_DIR, "c55_parameter_" + str(fx) + ".pth")
        extract.load_state_dict(torch.load(WEIGHTS_FILE))
        val = eTrain.test_single(extract, test_data, fx)
        print(f"Predected {EFFECT_MAP[fx]} has value {val}")
        eff.addEffect(fx, val, mode="manual")

waveform, sr = torchaudio.load(os.path.join(AUDIO_DIR, sample_file))

play_audio(waveform, sr)
play_audio(eff.generate(), sr)

# table = pd.DataFrame(log)
# print(table)


Test file: c54_7_od_1_cs_0_tr_1_dl_2.wav, has distortion with value 0.53, has chorus with value 0.23, has tremolo with value 0.56, has delay with value 0.88
Predected distortion has value 0.54
Predected chorus has value 0.17
Predected tremolo has value 0.67
Predected delay has value 0.84
